In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from openai import AzureOpenAI
AzureOpenAI.api_key = os.getenv("AZURE_OPENAI_API_KEY")

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_openai import AzureChatOpenAI
from langchain_groq import ChatGroq

llm_groq = ChatGroq(api_key=groq_api_key, model="llama-3.1-8b-instant")
llm_groq

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x7f9860784e50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f98606e5610>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following English text to French."),
    HumanMessage(content="Hello, how are you?")
]

messages

[SystemMessage(content='Translate the following English text to French.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})]

In [5]:
result = llm_groq.invoke(messages)

In [6]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

parsed_output = output_parser.invoke(result)

In [7]:
parsed_output

'Bonjour, comment allez-vous ? \n\nHere\'s a breakdown:\n\n- "Hello" is translated to "Bonjour" (formal) or "Salut" (informal)\n- "How are you?" is translated to "Comment allez-vous?" (formal) or "Comment ça va?" (informal)'

In [8]:
## Using LCEL (LangChain Expression Language) - chain the components together

chain = llm_groq | output_parser
chain.invoke(messages)

'Bonjour, comment allez-vous ?'

In [11]:
## Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following English text to {language}."

promptTemplate = ChatPromptTemplate.from_messages([
    ("system", generic_template),
    ("user", "{text}")
])

In [12]:
promptTemplate.invoke({"language": "German", "text": "Hello, how are you?", })

ChatPromptValue(messages=[SystemMessage(content='Translate the following English text to German.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you?', additional_kwargs={}, response_metadata={})])

In [13]:
chain = promptTemplate | llm_groq | output_parser
chain.invoke({"language": "Spanish", "text": "Good morning!"})

'Buenos días.'